# Glyph Manifold Simulation Notebook
Companion to `glyph_manifold_v1.0.md` — reproduces the coherence experiments:
- H₂O symbolic molecule, chaos, stabilizers
- Coherence metric over time
- Optional CH₄ variant


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

nx = ny = 200
x = np.linspace(-5, 5, nx)
y = np.linspace(-5, 5, ny)
X, Y = np.meshgrid(x, y)
dx = x[1] - x[0]
dt = 0.01
steps = 300
lam = 0.5

def laplacian(Z):
    return (np.roll(Z,1,0)+np.roll(Z,-1,0)+np.roll(Z,1,1)+np.roll(Z,-1,1)-4*Z)/(dx*dx)

def coherence(Phi):
    theta = np.angle(Phi)
    gx, gy = np.gradient(theta, dx, edge_order=2)
    return np.mean(1.0/(1.0 + gx*gx + gy*gy))

def radial_source(freq, x0, y0):
    R = np.sqrt((X-x0)**2 + (Y-y0)**2)
    return np.sin(freq*R) * np.exp(-R)

f_H, f_O = 0.45, 2.10
O = radial_source(f_O, 0.0, 0.0)
H1 = radial_source(f_H, -1.5, 1.0)
H2 = radial_source(f_H,  1.5, 1.0)

rng = np.random.default_rng(42)
chaos = np.zeros_like(X, dtype=float)
for _ in range(5):
    cx, cy = rng.uniform(-4,4,2)
    chaos += radial_source(6.66, cx, cy)

stab = (radial_source(0.70,  0.0,-2.0)
      + radial_source(0.60, -3.0,-3.0)
      + radial_source(5.85,  3.0,-3.0))

Phi = np.exp(-(X**2+Y**2)) * np.exp(1j*0.0)
C_hist = []
for t in range(steps):
    S = 0.2*(O + H1 + H2) + 0.15*chaos + 0.2*stab
    dPhi = laplacian(Phi) - lam*(np.abs(Phi)**2 - 1.0)*Phi + S
    Phi = Phi + dt*dPhi
    C_hist.append(coherence(Phi))

plt.figure(figsize=(5,5))
plt.imshow(np.abs(Phi), extent=[x.min(), x.max(), y.min(), y.max()], origin='lower')
plt.title('Final Field |Phi| (H2O + Chaos + Stabilizers)')
plt.axis('off')
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,4))
plt.plot(C_hist)
plt.title('Coherence vs. Time')
plt.xlabel('Step')
plt.ylabel('C(t)')
plt.tight_layout()
plt.show()


## Optional: CH₄ (Methane) Variant

In [ ]:
Phi = np.exp(-(X**2+Y**2)) * np.exp(1j*0.0)
C_hist2 = []
f_C = 1.80
C0 = radial_source(f_C, 0.0, 0.0)
H_up    = radial_source(f_H, 0.0,  1.5)
H_down  = radial_source(f_H, 0.0, -1.5)
H_left  = radial_source(f_H,-1.5,  0.0)
H_right = radial_source(f_H, 1.5,  0.0)
for t in range(steps):
    S = 0.2*(C0 + H_up + H_down + H_left + H_right) + 0.15*chaos + 0.2*stab
    dPhi = laplacian(Phi) - lam*(np.abs(Phi)**2 - 1.0)*Phi + S
    Phi = Phi + dt*dPhi
    C_hist2.append(coherence(Phi))

plt.figure(figsize=(5,5))
plt.imshow(np.abs(Phi), extent=[x.min(), x.max(), y.min(), y.max()], origin='lower')
plt.title('Final Field |Phi| (CH4 + Chaos + Stabilizers)')
plt.axis('off')
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,4))
plt.plot(C_hist2)
plt.title('Coherence vs. Time (CH4 experiment)')
plt.xlabel('Step')
plt.ylabel('C(t)')
plt.tight_layout()
plt.show()
